Reference: https://wiki.personaldata.io/w/api.php?action=help&modules=expandtemplates

In [74]:
import requests
import urllib.parse
import pandas as pd
import json

In [75]:
# manual list of “static” properties
properties = ['Q988', 'Q1010', 'Q1011', 'Q1243', 'Q101', 'Q241', 'Q1022', 'Q1938', 'Q252', 'Q3345', 'Q1185', 'Q3531']

In [76]:
# to query wiki.personaldata.io for 1 item

def getText(q):
    data = {'key': q, 'data': None}
    endpoint = 'https://wiki.personaldata.io/w/api.php'

    load = {
        'action': 'expandtemplates',
        'text': '{{MailtoSwissAccess|' + q + '}}',
        'format': 'json',
        'prop': 'wikitext'
    }
    try:
        response = requests.post(endpoint, data=load)
    except requests.exceptions.RequestException as e: # on catch tout
        print (e)
        return False
    if response:
        data['data'] = response.json()['expandtemplates']['wikitext']
    return data

In [77]:
items = []

In [78]:
for prop in properties:
    items.append(getText(prop))

In [79]:
df = pd.DataFrame(items)

In [80]:
def getRecipient(text):
    mailto, subject = urllib.parse.parse_qsl(text)[0]
    return subject.replace("Demande d'accès", "").strip()

def getMailto(text):
    mailto, subject = urllib.parse.parse_qsl(text)[0]
    return mailto.split('?')[0]
df['Recipient'] = df['data'].apply(getRecipient)

In [81]:
df['Mailto'] = df['data'].apply(getMailto)

In [82]:
df['Text-unquoted'] = df['data'].apply(lambda x: urllib.parse.unquote(x).split('&body=')[1])

In [83]:
df['subject'] = df['Text-unquoted'].apply(lambda x: x.split('\n\n')[0])

In [84]:
df['message'] = df.apply(lambda row: row['Text-unquoted'].replace(row['subject'], '').strip(), axis=1)

In [85]:
df['recipient'] = df['Mailto'].apply(lambda x: x.replace('mailto:', ''))

In [86]:
df.sort_values('Recipient', inplace=True)

In [87]:
jsondata = {'data': df.to_dict(orient='records')}

In [88]:
with open('data/companies.json', 'w') as fp:
    json.dump(jsondata, fp)

In [89]:
df

,key,data,Recipient,Mailto,Text-unquoted,subject,message,recipient
3,Q1243,mailto:dpo@blizzard.com?subject=Demande%20d%27...,Blizzard Entertainment,mailto:dpo@blizzard.com,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Blizzard Entertainment,\n\nMadame, Monsie...",dpo@blizzard.com
7,Q1938,mailto:datenschutz@sbb.ch?subject=Demande%20d%...,CFF,mailto:datenschutz@sbb.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher CFF,\n\nMadame, Monsieur,\n\nPar la prése...",datenschutz@sbb.ch
5,Q241,mailto:datarequests@support.facebook.com?subje...,Facebook,mailto:datarequests@support.facebook.com,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Facebook,\n\nMadame, Monsieur,\n\nPar la ...",datarequests@support.facebook.com
9,Q3345,mailto:datenschutz@intrum.ch?subject=Demande%2...,Intrum Justitia,mailto:datenschutz@intrum.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Intrum Justitia,\n\nMadame, Monsieur,\n\n...",datenschutz@intrum.ch
10,Q1185,mailto:protectiondonnees@letemps.ch?subject=De...,Le Temps,mailto:protectiondonnees@letemps.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Le Temps,\n\nMadame, Monsieur,\n\nPar la ...",protectiondonnees@letemps.ch
1,Q1010,mailto:postfinance@postfinance.ch?subject=Dema...,PostFinance,mailto:postfinance@postfinance.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher PostFinance,\n\nMadame, Monsieur,\n\nPar ...",postfinance@postfinance.ch
11,Q3531,mailto:datenschutz@ringieraxelspringer.ch?subj...,Ringier Axel Springer Switzerland SA,mailto:datenschutz@ringieraxelspringer.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Ringier Axel Springer Switzerland SA,\n\n...",datenschutz@ringieraxelspringer.ch
0,Q988,mailto:datenschutz@swisscom.com?subject=Demand...,Swisscom,mailto:datenschutz@swisscom.com,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Swisscom,\n\nMadame, Monsieur,\n\nPar la ...",datenschutz@swisscom.com
6,Q1022,mailto:privacy@gotinder.com?subject=Demande%20...,Tinder,mailto:privacy@gotinder.com,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Tinder,\n\nMadame, Monsieur,\n\nPar la pr...",privacy@gotinder.com
2,Q1011,mailto:twint@twint.ch?subject=Demande%20d%27ac...,Twint,mailto:twint@twint.ch,Demande d'accès aux données personnelles (art....,Demande d'accès aux données personnelles (art....,"Cher Twint,\n\nMadame, Monsieur,\n\nPar la pré...",twint@twint.ch
